In [3]:
import pandas as pd

# Cargar los dataframes
df1_path = r"C:\Users\juan_garnicac\Documents\post\Definición_Categorias_TikTok.xlsx"
df2_path = r"C:\Users\juan_garnicac\Downloads\soloComentariosUnicosTikTok.xlsx"

df1 = pd.read_excel(df1_path, sheet_name='Trabajo14Octubre')
df2 = pd.read_excel(df2_path, sheet_name='Hoja1')

# Verificar si tienen columna "Post" (con mayúscula)
print("=== VERIFICACIÓN COLUMNA 'Post' ===")
print(f"DataFrame 1 tiene columna 'Post': {'Post' in df1.columns}")
print(f"DataFrame 2 tiene columna 'Post': {'Post' in df2.columns}")

# Si existen, mostrar información específica de esas columnas
if 'Post' in df1.columns:
    print(f"\nDataFrame 1 - Columna 'Post':")
    print(f"  Tipo: {df1['Post'].dtype}")
    print(f"  Valores únicos: {df1['Post'].nunique()}")
    print(f"  Primeros valores: {df1['Post'].head().tolist()}")
    print(f"  ¿Tiene nulos?: {df1['Post'].isnull().sum()}")
    print(f"  Muestra de valores:")
    print(df1['Post'].head(10))

if 'Post' in df2.columns:
    print(f"\nDataFrame 2 - Columna 'Post':")
    print(f"  Tipo: {df2['Post'].dtype}")
    print(f"  Valores únicos: {df2['Post'].nunique()}")
    print(f"  Primeros valores: {df2['Post'].head().tolist()}")
    print(f"  ¿Tiene nulos?: {df2['Post'].isnull().sum()}")
    print(f"  Muestra de valores:")
    print(df2['Post'].head(10))

# Mostrar todas las columnas para referencia
print(f"\n=== TODAS LAS COLUMNAS ===")
print(f"DataFrame 1: {df1.columns.tolist()}")
print(f"DataFrame 2: {df2.columns.tolist()}")

# Mostrar formas de los dataframes
print(f"\n=== FORMAS ===")
print(f"DataFrame 1: {df1.shape} (filas, columnas)")
print(f"DataFrame 2: {df2.shape} (filas, columnas)")

# Verificar si hay valores comunes en la columna Post
if 'Post' in df1.columns and 'Post' in df2.columns:
    post_common = set(df1['Post'].dropna()) & set(df2['Post'].dropna())
    print(f"\n=== VALORES COMUNES EN 'Post' ===")
    print(f"Número de valores comunes: {len(post_common)}")
    if len(post_common) > 0:
        print(f"Algunos valores comunes: {list(post_common)[:5]}")

=== VERIFICACIÓN COLUMNA 'Post' ===
DataFrame 1 tiene columna 'Post': True
DataFrame 2 tiene columna 'Post': True

DataFrame 1 - Columna 'Post':
  Tipo: object
  Valores únicos: 176
  Primeros valores: ['?? ¡La serie de tu vida comienza en la CUN!  ?? Con CT AYUDA, solo necesitas el 10% de la matrícula inicial. ¡Nosotros financiamos el resto!', '?? ¡La serie de tu vida comienza en la CUN!  ?? Con CT AYUDA, solo necesitas el 10% de la matrícula inicial. ¡Nosotros financiamos el resto!', '?? ¡La serie de tu vida comienza en la CUN!  ?? Con CT AYUDA, solo necesitas el 10% de la matrícula inicial. ¡Nosotros financiamos el resto!', '?? ¡La serie de tu vida comienza en la CUN!  ?? Con CT AYUDA, solo necesitas el 10% de la matrícula inicial. ¡Nosotros financiamos el resto!', '?? ¿Y para ti, qué canción representa ese momento tan especial? ????  #grados #grado #sueñosymetas']
  ¿Tiene nulos?: 0
  Muestra de valores:
0    ?? ¡La serie de tu vida comienza en la CUN!  ?...
1    ?? ¡La serie de tu

In [ ]:
import pandas as pd
import tabulate as tb
import re
import unicodedata

# Cargar los dataframes
df1_path = r"C:\Users\juan_garnicac\Documents\post\Definición_Categorias_TikTok.xlsx"
df2_path = r"C:\Users\juan_garnicac\Downloads\soloComentariosUnicosTikTok.xlsx"

df1 = pd.read_excel(df1_path, sheet_name='Trabajo14Octubre')
df2 = pd.read_excel(df2_path, sheet_name='Hoja1')

# Función para normalizar texto completamente
def normalizar_texto(texto):
    if pd.isna(texto):
        return ""
    
    # Convertir a string y minúsculas
    texto = str(texto).lower().strip()
    
    # Eliminar tildes y caracteres especiales
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    
    # Eliminar signos de puntuación y caracteres especiales
    texto = re.sub(r'[^\w\s]', '', texto)
    
    # Reemplazar múltiples espacios por uno solo
    texto = re.sub(r'\s+', ' ', texto)
    
    return texto.strip()

# Aplicar normalización a la columna Post en ambos dataframes
print("Normalizando textos...")
df1['Post_normalizado'] = df1['Post'].apply(normalizar_texto)
df2['Post_normalizado'] = df2['Post'].apply(normalizar_texto)

# Verificar coincidencias después de la normalización
coincidencias = set(df1['Post_normalizado']) & set(df2['Post_normalizado'])
print(f"Coincidencias encontradas después de normalizar: {len(coincidencias)}")

# Mostrar algunas coincidencias normalizadas
if len(coincidencias) > 0:
    print("\nAlgunas coincidencias normalizadas:")
    for i, post in enumerate(list(coincidencias)[:3]):
        print(f"{i+1}. {post[:100]}...")

# Seleccionar las columnas de df2 que queremos llevar al merge
columnas_df2 = ['Post_normalizado', 'Categoría', 'Subcategoría', 'Trigger', 'Unnamed: 4']

# Hacer el merge con los textos normalizados
merged_df = pd.merge(df1, df2[columnas_df2], on='Post_normalizado', how='left')

# Eliminar la columna normalizada del resultado final
merged_df = merged_df.drop('Post_normalizado', axis=1)

# Ruta para guardar el archivo
output_path = r"C:\Users\juan_garnicac\Documents\post\mergTiktoK.xlsx"

# Guardar el DataFrame mergeado en Excel
merged_df.to_excel(output_path, index=False)

print("\n=== ARCHIVO GUARDADO EXITOSAMENTE ===")
print(f"📊 DataFrame principal (df1): {df1.shape}")
print(f"📊 DataFrame secundario (df2): {df2.shape}")
print(f"✅ DataFrame mergeado: {merged_df.shape}")
print(f"🎯 Coincidencias encontradas: {len(coincidencias)} posts únicos")

# Calcular filas con categorías asignadas
filas_con_categoria = merged_df[~merged_df['Categoría'].isna()].shape[0]
porcentaje = (filas_con_categoria / len(merged_df)) * 100
print(f"📈 Filas con categorías asignadas: {filas_con_categoria} de {len(merged_df)} ({porcentaje:.2f}%)")
print(f"💾 Archivo guardado en: {output_path}")

# Mostrar resumen por categorías
if not merged_df['Categoría'].isna().all():
    print(f"\n=== DISTRIBUCIÓN POR CATEGORÍAS ===")
    categorias_count = merged_df['Categoría'].value_counts()
    print(tb.tabulate(categorias_count.reset_index(), 
                     headers=['Categoría', 'Cantidad'], 
                     tablefmt='grid'))

# Mostrar ejemplos de resultados
print(f"\n=== EJEMPLOS DE RESULTADOS ===")

# Filas CON coincidencia
if filas_con_categoria > 0:
    print("\n🧩 Filas CON categorías asignadas:")
    con_categoria = merged_df[~merged_df['Categoría'].isna()].head(2)
    print(tb.tabulate(con_categoria, headers='keys', tablefmt='grid', showindex=False, maxcolwidths=50))

# Verificación de que el archivo se guardó correctamente
import os
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / 1024  # Tamaño en KB
    print(f"\n✅ Verificación: Archivo creado exitosamente")
    print(f"📏 Tamaño del archivo: {file_size:.2f} KB")
else:
    print("\n❌ Error: El archivo no se pudo crear")

Normalizando textos...
Coincidencias encontradas después de normalizar: 175

Algunas coincidencias normalizadas:
1. cumple tus metas academicas con credipoli ya no hay obstaculos para alcanzar tu sueno de estudiar en...
2. areandina es tu mejor opcion matriculate ya aprovecha los descuentos y las multiples opciones de fin...
3. la serie de tu vida comienza en la cun con ct ayuda solo necesitas el 10 de la matricula inicial nos...

=== ARCHIVO GUARDADO EXITOSAMENTE ===
📊 DataFrame principal (df1): (3736, 2)
📊 DataFrame secundario (df2): (177, 6)
✅ DataFrame mergeado: (3736, 5)
🎯 Coincidencias encontradas: 175 posts únicos
📈 Filas con categorías asignadas: 3732 de 3736 (99.89%)
💾 Archivo guardado en: C:\Users\juan_garnicac\Documents\post\mergTiktoK.xlsx

=== DISTRIBUCIÓN POR CATEGORÍAS ===
+----+---------------------------------------+------------+
|    | Categoría                             |   Cantidad |
+====+=======================================+============+
|  0 | Entretenimient

: 